# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
#!pip install pandas

In [18]:
import pandas as pd
df = pd.read_csv('/Users/rawan/Desktop/git_projects/lab-chains-in-langchain/data/Data.csv')

In [19]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [20]:
!pip install langchain_community

In [21]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [31]:
#Replace None by your own value and justify
#Optimal for tasks requiring balanced creativity and coherence, this setting introduces moderate randomness while maintaining factual reliability.
llm = ChatOpenAI(temperature=.5 )



In [32]:
prompt = ChatPromptTemplate.from_template( #Write a query that would take a variable to describe any product
  """Write a detailed product description for a {product_name}: 
    - Target audience: {target_audience}
    - Key features: {features}
    
    
    """
)


In [33]:

chain = LLMChain(llm=llm, prompt=prompt)

In [34]:
product = {
    
    "product_name": "SmartGrow Indoor Garden Kit",
   
    "target_audience": "people who have garden in their house and gardening enthusiasts",
    "features": "automated watering system, LED grow lights, app integration"
    
    
}
chain.run(product)

"The SmartGrow Indoor Garden Kit is the perfect solution for individuals who love gardening but may not have the time or space for a traditional outdoor garden. This innovative kit is designed for indoor use, making it ideal for those who live in apartments, condos, or homes with limited outdoor space.\n\nOne of the key features of the SmartGrow Indoor Garden Kit is its automated watering system. This system takes the guesswork out of watering your plants, ensuring they receive the perfect amount of water at the right time. This not only saves you time and effort, but also helps to prevent over or under-watering, which can be detrimental to plant health.\n\nThe kit also comes equipped with LED grow lights, which provide the perfect spectrum of light for plants to thrive. These lights mimic natural sunlight, promoting healthy growth and ensuring that your plants receive the light they need, even in a dimly lit indoor environment.\n\nFor added convenience, the SmartGrow Indoor Garden Kit

## SimpleSequentialChain

In [35]:
from langchain.chains import SimpleSequentialChain

In [53]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    #Repeat the initial query or create a new query that would feed into the second prompt
      """Write a detailed product description for a {product_name}: 
    - Target audience: {target_audience}
    - Key features: {features}
    
    
    """
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key='product_description')



In [54]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
     """Refine and expand this product description for better marketing appeal. 
    Focus on enhancing {features} while preserving the core details:
    
    **Original Description**: {product_description}
    
    **Revised Version**:"""
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [56]:
from langchain.chains import SequentialChain

# Combined chain
overall_chain = SequentialChain(
    chains=[chain_one, chain_two],
    input_variables=['product_name', 'target_audience', 'features'],
    output_variables=['product_description'],  # adjust if the second chain outputs a different name
    verbose=True
)


In [57]:
overall_chain.run(product)



> Entering new SequentialChain chain...

> Finished chain.


"The SmartGrow Indoor Garden Kit is the perfect solution for gardeners who want to bring their green thumb indoors. This innovative kit is designed for people who have a garden in their house or are gardening enthusiasts looking to grow their favorite plants year-round.\n\nOne of the key features of the SmartGrow Indoor Garden Kit is its automated watering system. This system ensures that your plants are provided with the perfect amount of water at the right times, helping to prevent overwatering or underwatering. This feature is especially helpful for busy individuals who may not always have the time to monitor their plants' watering needs.\n\nAdditionally, the SmartGrow Indoor Garden Kit comes equipped with high-quality LED grow lights, which mimic natural sunlight and promote healthy plant growth. These lights are energy-efficient and have a long lifespan, making them a cost-effective and environmentally-friendly option for indoor gardening.\n\nThe kit also features app integration,

**Repeat the above twice for different products**

In [58]:
product1 = {
    "product_name": "CozyHeat Electric Blanket",
   
    "target_audience": "People living in cold climates and comfort seekers",
    "features": [
        "Adjustable heat settings",
        "Auto shut-off timer",
        "Machine washable fabric"
    ],
    
    "rating": 4.6,
    "reviews_count": 2350
}

chain.run(product1)


"Stay warm and cozy all winter long with the CozyHeat Electric Blanket. Perfect for people living in cold climates or those who simply love to snuggle up in comfort, this electric blanket is sure to become your new favorite way to stay warm.\n\nThe CozyHeat Electric Blanket features adjustable heat settings, allowing you to customize the level of warmth to suit your preferences. Whether you like a little extra heat on a chilly night or just need a gentle warmth to help you relax, this blanket has you covered. The auto shut-off timer ensures safety and peace of mind, so you can drift off to sleep without worrying about leaving the blanket on all night.\n\nMade with machine washable fabric, the CozyHeat Electric Blanket is easy to care for and keep clean. Simply toss it in the washing machine when it needs a refresh and enjoy the soft, cozy feel of the blanket for years to come.\n\nDon't let the cold weather get you down – stay warm and comfortable with the CozyHeat Electric Blanket. Per

In [59]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_pro = ChatPromptTemplate.from_template(
    #Repeat the initial query or create a new query that would feed into the second prompt
      """Write a detailed product description for a {product_name}: 
    - Target audience: {target_audience}
    - Key features: {features}
    - Rating: {rating}
    - Number of reviews: {reviews_count}
    
    
    """
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key='product_description')



In [64]:

# prompt template 2
second_pro = ChatPromptTemplate.from_template(
    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
     """Refine and expand this product description for better marketing appeal. 
    Focus on enhancing {features} while preserving the core details:
    
    **Original Description**: {product_description}
    
    **Revised Version**:"""
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [65]:
from langchain.chains import SequentialChain

# Combined chain
overall_chain1 = SequentialChain(
    chains=[chain_one, chain_two],
    input_variables=['product_name', 'target_audience', 'features', 'rating', 'reviews_count'],
    output_variables=['product_description'],  # adjust if the second chain outputs a different name
    verbose=True
)


In [66]:
overall_chain1.run(product1)



> Entering new SequentialChain chain...

> Finished chain.


"The CozyHeat Electric Blanket is the perfect solution for those living in cold climates or for anyone who loves to snuggle up and stay warm and cozy. This luxurious blanket features adjustable heat settings, allowing you to customize the warmth to your preferred level. Whether you like a little extra heat on chilly nights or just a gentle warmth to keep you comfortable, this blanket has you covered.\n\nOne of the key features of the CozyHeat Electric Blanket is the auto shut-off timer, providing peace of mind and safety. You can set the timer to automatically turn off after a certain amount of time, so you can sleep soundly without worrying about overheating or leaving the blanket on all night.\n\nThe machine washable fabric makes cleaning and maintenance a breeze. Simply toss the blanket in the washing machine to keep it fresh and clean, ensuring that you can enjoy the comfort and warmth of this electric blanket for years to come.\n\nWhether you're curling up on the couch for a movie

## SequentialChain

In [67]:
from langchain.chains import SequentialChain

In [74]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  #This prompt should translate a review
   "Translate the following customer review to English:\n\n{review}"

)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key= "translated_review" #Give a name to your output
                    )


In [75]:
second_prompt = ChatPromptTemplate.from_template(
    #Write a promplt to summarize a review
     "Summarize the following customer review in 1-2 sentences:\n\n{review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key= "summary" #give a name to this output
                    )


In [76]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
     "Translate the following customer review to Arabic:\n\n{review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="translate"
                      )


In [77]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Based on the translated review:\n\n{translated_review}\n\n"
    "and the summary:\n\n{summary}\n\n"
    "Write a friendly follow-up message to the customer thanking them for their feedback!"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="follow_up_message"
                     )


In [81]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables= ["review"],
    output_variables=["translated_review", "translate","summary", "follow_up_message"],
    verbose=True
)

In [82]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'translated_review': '"I find the taste mediocre. The foam doesn\'t hold, it\'s weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?"',
 'translate': 'أجد الطعم سيئًا. لا يدوم الرغوة، هذا أمر غريب. أشتري نفس الشيء من المتاجر والطعم أفضل بكثير...\nهل هو دفعة قديمة أم تقليد؟',
 'summary': 'Le client trouve le goût médiocre, la mousse ne tient pas, et pense que le produit pourrait être un vieux lot ou une contrefaçon car il achète habituellement le même produit dans le commerce avec un meilleur goût.',
 'follow_up_message': 'Dear Valued Customer,\n\nThank you for taking the time to share your feedback with us. We are sorry to hear that you found the taste of our product to be mediocre and that the foam did not hold up as expected. We take customer feedback very seriously

**Repeat the above twice for different products or reviews**

## Router Chain

In [84]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [85]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [86]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [87]:
llm = ChatOpenAI(temperature=0)

In [88]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [89]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [90]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [91]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [92]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [93]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [94]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [95]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This genetic information is crucial for maintaining the integrity and stability of our cells and ultimately our entire body. \n\nAdditionally, DNA is constantly being replicated and repaired in our cells to ensure that genetic information is passed on accurately to the next generation of cells. This process is essential for the growth, development, and overall health of our bodies. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and maintenance of a

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [112]:
therapist_template = """You are a compassionate therapist.
You have a deep understanding of human emotions, behaviors, and mental health.
You are great at creating a safe space for individuals to explore their feelings,
challenges, and personal growth. You excel at listening, offering empathetic responses,
and helping people work through difficult emotions. You offer guidance in a supportive
and non-judgmental way, helping people find their own solutions and gain new perspectives.

Here is a question: {input}"""

art_template = """You are an imaginative and insightful artist. You see the world through a creative lens, finding beauty and meaning in both the ordinary and the extraordinary. You believe in the power of art to express complex emotions, ideas, and stories without words. You encourage exploration and experimentation, helping individuals tap into their creative potential and discover their unique artistic voice. Your approach to art is fluid, allowing for self-expression and the exploration of different mediums, styles, and techniques.

Here is a question: {input}"""


In [113]:
p_infos = [
    {
        "name": "therapy",
        "description": "Good for answering questions in a therapeutic context",
        "prompt_template": therapist_template
    },
    {
    "name": "art",
    "description": "Good for answering art-related questions and promoting creativity",
    "prompt_template": art_template
}

]

In [114]:
llm = ChatOpenAI(temperature=0.3)

In [116]:
destination_chains = {}
for p_info in p_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in p_infos]
destinations_str = "\n".join(destinations)


In [117]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [123]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [124]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [125]:
chain1 = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [127]:
chain1.run("I tend to overthink things. How can I break out of this cycle?")



> Entering new MultiPromptChain chain...
therapy: {'input': 'I tend to overthink things. How can I break out of this cycle?'}
> Finished chain.


"It's great that you're recognizing this pattern of overthinking and seeking help to break out of it. Overthinking can be exhausting and can lead to increased anxiety and stress. Here are some strategies that may help you break out of this cycle:\n\n1. Practice mindfulness: Mindfulness involves being present in the moment and focusing on your thoughts and feelings without judgment. This can help you become more aware of when you're overthinking and bring your attention back to the present moment.\n\n2. Challenge negative thoughts: When you catch yourself overthinking, try to challenge the negative thoughts that are fueling the cycle. Ask yourself if there is evidence to support these thoughts or if there are alternative, more positive interpretations.\n\n3. Set boundaries with your thoughts: Give yourself permission to let go of thoughts that are not serving you. Set aside specific times during the day to address your concerns and worries, and then let them go until the next designated

In [128]:
chain1.run("How can I develop healthier coping mechanisms for when I feel stressed or overwhelmed?")



> Entering new MultiPromptChain chain...
therapy: {'input': 'How can I develop healthier coping mechanisms for when I feel stressed or overwhelmed?'}
> Finished chain.


"Developing healthier coping mechanisms for stress and overwhelm is an important aspect of self-care and mental well-being. Here are some suggestions that may help you in this process:\n\n1. Identify your triggers: Pay attention to what situations or thoughts tend to trigger your stress or overwhelm. By identifying these triggers, you can work on addressing them more effectively.\n\n2. Practice mindfulness: Mindfulness techniques, such as deep breathing, meditation, or yoga, can help you stay present and calm in the face of stress. These practices can also help you become more aware of your thoughts and emotions.\n\n3. Engage in physical activity: Exercise is a great way to release pent-up stress and tension. Whether it's going for a walk, practicing yoga, or hitting the gym, physical activity can help improve your mood and reduce stress levels.\n\n4. Seek support: Talking to a trusted friend, family member, or therapist can provide you with emotional support and help you gain new pers

In [132]:
llm = ChatOpenAI(temperature=0.9)

In [133]:
destination_chains = {}
for p_info in p_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in p_infos]
destinations_str = "\n".join(destinations)


In [134]:
chain2 = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [135]:
chain2.run("Can you tell me a joke?")




> Entering new MultiPromptChain chain...
None: {'input': 'Can you tell me a joke?'}
> Finished chain.


"Sure, here's one for you:\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two tired!"

In [137]:
chain2.run("short story about a ghost and a sea?")



> Entering new MultiPromptChain chain...
art: {'input': 'short story about a ghost and a sea?'}
> Finished chain.


"The sea was calm and serene, its waves gently lapping against the shore as the sun dipped below the horizon. The ghost of a young girl stood on the sandy beach, her translucent form shimmering in the fading light. She had been haunting this stretch of coastline for decades, unable to move on to the afterlife.\n\nThe ghost wandered along the water's edge, her ethereal form leaving no footprints in the sand. She longed to feel the cool touch of the ocean, but her ghostly nature prevented her from fully experiencing its beauty. As she gazed out at the vast expanse of water, memories flooded her mind of a time long ago when she was alive and free.\n\nSuddenly, a mist began to form on the horizon, swirling and coalescing into a figure of pure white light. The ghost watched in awe as the specter of a sea spirit emerged from the depths, its form ever-changing and shimmering like the waves themselves.\n\nThe sea spirit approached the ghost, its presence both comforting and unsettling. It spok

In [138]:
chain2.run("ideas of a painting that represents the concept of time?")  




> Entering new MultiPromptChain chain...
art: {'input': 'ideas of a painting that represents the concept of time?'}
> Finished chain.


'One idea for a painting that represents the concept of time could be a series of interconnected clocks, each showing a different time of day or representing a different moment in history. The clocks could be melting or distorted, symbolizing the fluidity and impermanence of time. Another idea could be a figure or object caught in a perpetual state of movement, with blurred lines and overlapping images to convey the passing of time. The painting could also incorporate symbols such as hourglasses, sundials, or calendars to further emphasize the theme of time. Ultimately, the painting would aim to visually capture the abstract concept of time and its impact on our lives.'

In [140]:
chain2.run("Can you give me a summary of the book 'Harry Potter'?")



> Entering new MultiPromptChain chain...
None: {'input': "Can you give me a summary of the book 'Harry Potter'?"}
> Finished chain.


'Sure! "Harry Potter" is a series of fantasy novels written by J.K. Rowling that follows the life of a young wizard named Harry Potter. The story begins with Harry discovering that he is a wizard on his eleventh birthday and is accepted into Hogwarts School of Witchcraft and Wizardry. Throughout the series, Harry learns about his past, battles dark forces, and ultimately faces the dark wizard Voldemort. Along the way, he forms strong friendships with Ron Weasley and Hermione Granger, and experiences love, loss, and triumph. The series explores themes of friendship, bravery, and the power of love.'

In [141]:
chain2.run("create a new song about love?") 



> Entering new MultiPromptChain chain...
art: {'input': 'create a new song about love?'}
> Finished chain.


"Verse 1:\nLove is a melody that plays in my heart\nA symphony of emotions that sets me apart\nIt's a feeling so deep, it can't be denied\nA connection so strong, it can't be defied\n\nChorus:\nLove is a flame that never fades\nA light that guides us through the darkest of days\nIt's a bond that can never be broken\nA promise that will always be spoken\n\nVerse 2:\nLove is a canvas where we paint our dreams\nA journey of discovery, or so it seems\nIt's a dance of passion and desire\nA spark that ignites our inner fire\n\nChorus:\nLove is a flame that never fades\nA light that guides us through the darkest of days\nIt's a bond that can never be broken\nA promise that will always be spoken\n\nBridge:\nIn love, we find ourselves and each other\nIn love, we discover our true colors\nIn love, we embrace all that we are\nIn love, we shine like a distant star\n\nChorus:\nLove is a flame that never fades\nA light that guides us through the darkest of days\nIt's a bond that can never be broken\